광고 제거

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv('url_list_술집.csv', encoding='utf-8-sig')
data

,Unnamed: 0,0
0,0,https://blog.naver.com/jihey119/222224663683
1,1,https://blog.naver.com/belfry9/222204594293
2,2,https://blog.naver.com/sunhyeon4/222221712668
3,3,https://blog.naver.com/eksl139/222211391559
4,4,https://blog.naver.com/jamjma98/222193424549
...,...,...
2033,2033,https://blog.naver.com/ssul_lee/222221416740
2034,2034,https://blog.naver.com/goyha/222199677144
2035,2035,https://blog.naver.com/dlwlstjdehd1/222201936901
2036,2036,https://blog.naver.com/bsssum/222197529036


In [3]:
del data['Unnamed: 0']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2038 non-null   object
dtypes: object(1)
memory usage: 16.0+ KB


### 광고글 특징
1. 특정 단어 포함한 게시글 : '제공받아', '지원받아','해당업체'
2. (네이버 블로그만의 특징) 게시글 길이가 3000자 이상이면 광고글 상위 노출 조건에 해당하여 광고로 볼 수 있음

1번

In [ ]:
# 광고글 특징을 stopwords로 지정하여 해당 글은 삭제할 수 있게 함
# 분석가 판단 하에 추가 광고 문구 확인하여 stopwords에 추가

Stopwords = ['제공받아','제공받았','지원받아','지원받았','해당업체','해시태그','업체로부터',
             '체험후기','무상지급받은','무상지급','카카오톡플러스친구','톡톡하세요','네이버톡톡',
             '네이버 톡톡','무상지원','카카오톡 플러스친구','스토리링크','카카오 채널']


In [ ]:
data = data[~data['review'].str.contains('|'.join(Stopwords))]
data = data.reset_index(drop=True)
data

2번

In [ ]:
ix = data['review'].str.len() > 3000
data = data.loc[~ix]
data = data.reset_index(drop=True)
data

In [ ]:
# 기본 전처리
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus


In [ ]:
# 블로그 본문, 댓글 합쳐 하나의 document로 만들기

data['doc'] = data['review'] + ' ' + data['comment']
data.head()

In [ ]:
tqdm.pandas()

data['doc'] = data['doc'].progress_apply(lambda x : clean_text(x))